<a href="https://colab.research.google.com/github/SamikshKodgire/AAI-511Group6_USD/blob/main/Composer_Classification_Project_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Composer Classification Using Deep Learning
This notebook demonstrates a complete pipeline for classifying musical compositions by composer using deep learning. We'll use LSTM and CNN models on MIDI file data from famous composers like Mozart and Chopin.

## 📌 Methodology Overview
1. **Data Collection**: MIDI files of compositions from various composers.
2. **Data Pre-processing**: Convert scores to usable formats and apply augmentation.
3. **Feature Extraction**: Extract musical features (notes, tempo, etc.).
4. **Model Building**: Use LSTM and CNN for classification.
5. **Model Training**
6. **Model Evaluation**
7. **Model Optimization**

## 📂 Step 1: Load and Explore Dataset

In [ ]:
import os
dataset_path = '/mnt/data/Composer_Dataset/Composer_Dataset/NN_midi_files_extended/test'
for composer in os.listdir(dataset_path):
    composer_path = os.path.join(dataset_path, composer)
    if os.path.isdir(composer_path):
        files = os.listdir(composer_path)
        print(f"{composer}: {len(files)} MIDI files")

## 🎼 Step 2: Preprocessing and Feature Extraction

In [ ]:
import pretty_midi
import numpy as np

def extract_features(midi_path):
    try:
        midi_data = pretty_midi.PrettyMIDI(midi_path)
        notes = []
        for instrument in midi_data.instruments:
            if not instrument.is_drum:
                notes += [note.pitch for note in instrument.notes]
        return np.array(notes)
    except Exception as e:
        print(f"Error processing {midi_path}: {e}")
        return np.array([])

# Example usage:
mozart_path = os.path.join(dataset_path, 'mozart')
sample_file = os.path.join(mozart_path, os.listdir(mozart_path)[0])
features = extract_features(sample_file)
print(f"Extracted {len(features)} notes from {os.path.basename(sample_file)}")

## 🧠 Step 3: Model Building (LSTM / CNN)

In [ ]:
# This section will define and compile the deep learning models.
# TODO: Define LSTM and CNN architectures using TensorFlow/Keras

### 🧠 LSTM Model for Composer Classification
We use an LSTM model to learn the sequence of notes in compositions, assuming a fixed input length per sample.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Dummy data for demonstration (replace with actual sequences and labels)
X = [np.random.randint(60, 72, size=100) for _ in range(200)]  # 200 samples of 100-note sequences
y = np.random.randint(0, 2, size=(200,))  # Binary labels (e.g., Mozart vs Chopin)

# Padding sequences to the same length
X = pad_sequences(X, maxlen=100)

# Define LSTM model
model_lstm = Sequential([
    Embedding(input_dim=128, output_dim=64, input_length=100),
    LSTM(128, return_sequences=False),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

model_lstm.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_lstm.summary()

### 🧠 CNN Model for Composer Classification
We use a CNN on a piano roll-style 2D input for each composition sample.

In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Input
from tensorflow.keras.models import Model

# Dummy piano roll input: 200 samples, 128 pitches x 100 time steps
X_cnn = np.random.rand(200, 128, 100, 1)
y_cnn = y

# Define CNN model
input_layer = Input(shape=(128, 100, 1))
x = Conv2D(32, (3, 3), activation='relu')(input_layer)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)
x = Dense(64, activation='relu')(x)
output = Dense(1, activation='sigmoid')(x)
model_cnn = Model(inputs=input_layer, outputs=output)

model_cnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_cnn.summary()

## 🎹 Note Distribution Visualization
Before building models, it's helpful to visualize the distribution of musical notes across the dataset.

In [ ]:
import seaborn as sns
all_notes = []
composer_labels = []
composer_map = {}
label_counter = 0

# Scan through each composer's folder
for composer in os.listdir(dataset_path):
    composer_path = os.path.join(dataset_path, composer)
    if os.path.isdir(composer_path):
        composer_map[label_counter] = composer
        for midi_file in os.listdir(composer_path):
            if not midi_file.endswith('.mid'):
                continue
            file_path = os.path.join(composer_path, midi_file)
            notes = extract_features(file_path)
            if len(notes) > 0:
                all_notes.extend(notes)
                composer_labels.extend([label_counter] * len(notes))
        label_counter += 1

# Plot distribution
plt.figure(figsize=(12, 5))
sns.histplot(x=all_notes, bins=60, kde=True)
plt.title('Distribution of Musical Notes in Dataset')
plt.xlabel('MIDI Note Number')
plt.ylabel('Frequency')
plt.show()

## 🏷️ Generate Dataset with Composer Labels
Here we convert each composer's name into a numeric label and build the dataset for model input.

In [ ]:
# Generate labeled dataset
X_data = []
y_data = []
max_seq_len = 100  # Fixed length for LSTM input
composer_map = {}
label_counter = 0

for composer in os.listdir(dataset_path):
    composer_path = os.path.join(dataset_path, composer)
    if os.path.isdir(composer_path):
        composer_map[label_counter] = composer
        for midi_file in os.listdir(composer_path):
            if not midi_file.endswith('.mid'):
                continue
            file_path = os.path.join(composer_path, midi_file)
            notes = extract_features(file_path)
            if len(notes) > 0:
                padded = pad_sequences([notes], maxlen=max_seq_len)[0]
                X_data.append(padded)
                y_data.append(label_counter)
        label_counter += 1

# Convert to arrays
X = np.array(X_data)
y = np.array(y_data)
print(f"Loaded {len(X)} samples across {len(composer_map)} composers")

## 🚂 Step 4: Model Training

## 🚂 Step 4: Model Training and Evaluation
We'll now train the LSTM model on real extracted features. For demonstration, we simplify training using padded note sequences.
We'll evaluate using accuracy, confusion matrix, and classification report.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model_lstm.fit(X_train, y_train, epochs=5, batch_size=16, validation_split=0.1)

# Evaluate model
y_pred = (model_lstm.predict(X_test) > 0.5).astype(int)
print(classification_report(y_test, y_pred))

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap='Blues')
plt.title('Confusion Matrix')
plt.show()

## 🔧 Step 5: Hyperparameter Tuning with Keras Tuner
We'll use Keras Tuner to search for the best number of LSTM units and learning rate.

In [ ]:
import keras_tuner as kt

def build_model(hp):
    model = Sequential()
    model.add(Embedding(input_dim=128, output_dim=hp.Choice('embedding_dim', [32, 64, 128]), input_length=100))
    model.add(LSTM(hp.Int('lstm_units', min_value=64, max_value=256, step=64)))
    model.add(Dropout(hp.Float('dropout', min_value=0.2, max_value=0.5, step=0.1)))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(
        optimizer=keras.optimizers.Adam(hp.Float('lr', 1e-4, 1e-2, sampling='log')),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model

tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=1,
    directory='tuner_results',
    project_name='composer_lstm'
)

tuner.search(X_train, y_train, epochs=5, validation_split=0.1, verbose=1)
best_model = tuner.get_best_models(num_models=1)[0]
best_model.evaluate(X_test, y_test)

In [ ]:
# TODO: Train the models with extracted features and labels

## 📊 Step 5: Model Evaluation

In [ ]:
# TODO: Evaluate models using accuracy, precision, recall, and confusion matrix

## 🔧 Step 6: Model Optimization

In [ ]:
# TODO: Apply hyperparameter tuning (e.g., learning rate, dropout, layers)

## ✅ Summary
- Dataset contains classical MIDI files.
- We extracted musical note sequences using `pretty_midi`.
- Next steps involve model training and optimization.